# Steps:

- Document Loading
- Text Preprocessing
- Embedding
- Storing

### Imports

impor

In [1]:
import re
import string
from nltk.corpus import stopwords
import numpy as np
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import pdfplumber
import os
import faiss
import pickle

/Users/jean-sebastiengaultier/Desktop/UChicago/Q4/Capstone-Mosaic/capstone_venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Function to deal with the formatting of our text, make sure we have a homogenious corpus (remove stopwords, lowercase, remove punctuation)

- Add Lemmatization (maybe)

In [2]:
# Preprocessing function
def preprocess_text(text):
    text = text.lower() # Lowercase the text
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text) # Remove punctuation
    stop_words = set(stopwords.words('english'))
    text = " ".join([word for word in text.split() if word not in stop_words]) # Remove stopwords
    return text

Function that takes the text from an entire document and then creates chunks of specific size. Maps sentences and chunks of text to a very vast multi dimensional vectorspace

- Check how the tokenizer works

In [3]:
# Initialize tokenizer for token-based chunking
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

# Function to chunk text based on tokens
def chunk_text_by_tokens(text, max_tokens=512):
    tokens = tokenizer.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokens[i:i+max_tokens]
        chunks.append(tokenizer.decode(chunk, skip_special_tokens=True)) 
    return chunks

Embedding model can be switched. We need to see which one is better. (question how do we test to see which one is better)

In [4]:
# Initialize text and image embedding models
text_embedder = SentenceTransformer('all-MiniLM-L6-v2')  # Text embeddings

# Function to extract and embed images from PDFs
def extract_text_and_images_with_preprocessing(pdf_path):
    text = ""

    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            # Extract text from page
            page_text = page.extract_text()
            if page_text:
                text += page_text  # Append to overall text

    # Preprocess and chunk the text
    text = preprocess_text(text)
    chunks = chunk_text_by_tokens(text, max_tokens=512)

    return chunks


In [5]:
def process_pdfs_with_chunking_and_image_embedding(data_folder):
    documents = []
    for filename in os.listdir(data_folder):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(data_folder, filename)
            chunks = extract_text_and_images_with_preprocessing(pdf_path)
            documents.append({
                'chunks': chunks,
                'filename': filename
            })
    return documents

# Folder containing your PDFs
data_folder = '../Data/'

# Process the PDFs with chunking and image embedding
documents = process_pdfs_with_chunking_and_image_embedding(data_folder)


Token indices sequence length is longer than the specified maximum sequence length for this model (47209 > 512). Running this sequence through the model will result in indexing errors


In [6]:
# Embed the chunks
def embed_chunks(documents):
    for document in documents:
        document['chunk_embeddings'] = [{'text': chunk, 'embedding': text_embedder.encode(chunk, convert_to_tensor=True)} for chunk in document['chunks']]
    return documents

# Embed the chunks of all PDFs
embedded_documents = embed_chunks(documents)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


### FAISS Vectordatabase

In [8]:

build_folder = '../Vectordatabase/'
if not os.path.exists(build_folder):
    os.makedirs(build_folder)

# Initialize FAISS index for text chunks and image embeddings
embedding_dim_text = 384  # Dimension of the sentence transformer for text

# Create separate FAISS indices for text and image embeddings
index_text = faiss.IndexFlatL2(embedding_dim_text)

# Flatten all chunk embeddings and image embeddings and store their metadata
all_chunk_embeddings = []
chunk_metadata = []

for document in embedded_documents:
    # Add text chunk embeddings to index
    for chunk in document['chunk_embeddings']:
        all_chunk_embeddings.append(chunk['embedding'].cpu().numpy())
        chunk_metadata.append({'filename': document['filename'], 'text': chunk['text']})
    

# Convert embeddings to numpy arrays
all_chunk_embeddings = np.array(all_chunk_embeddings)

# Add embeddings to FAISS indices
index_text.add(all_chunk_embeddings)

# Save FAISS indices and metadata
faiss.write_index(index_text, 'financial_docs_text_index.faiss')


with open('financial_chunks_metadata.pkl', 'wb') as f:
    pickle.dump(chunk_metadata, f)



## Tests

In [3]:
import pickle

In [4]:
metadata_path = 'financial_chunks_metadata.pkl'
# Load metadata (chunked document data)
with open(metadata_path, "rb") as f:
    index_metadata = pickle.load(f)

# Print the structure of index_metadata to understand its contents
print("Index Metadata Structure:")
print(index_metadata)  # Display the content

# Check the type of the metadata for better understanding
print(f"Type of index_metadata: {type(index_metadata)}")


Index Metadata Structure:
[{'filename': 'PTLO 2023 Q4 10K.pdf', 'text': 'united states securities exchange commission washington dc 20549 form 10k annual report pursuant section 13 15d securities exchange act 1934 fiscal year ended december 31 2023 transition report pursuant section 13 15d securities exchange act 1934 transition period commission file number 00140951 portillos inc exact name registrant specified charter delaware 871104304 state jurisdiction incorporation organization irs employer identification 2001 spring road suite 400 oak brook illinois 60523 address principal executive offices 630 9543773 registrant ’ s telephone number including area code securities registered pursuant section 12b act title class trading symbol name exchange registered class common stock 001 par value per share ptlo nasdaq global select market securities registered pursuant section 12g act none indicate check mark registrant wellknown seasoned issuer defined rule 405 securities act yes indicate ch